In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
#import plotly.express as px
import json


###Loading Data

In [ ]:
def load_data(path: str) -> [pd.DataFrame]:
  df = pd.read_csv(path,
                  header=None,
                  names=['id', 'translation'],
                  quotechar='"',
                  escapechar='\\',
                  delimiter='|')

  rows_to_drop = df[(df["id"] == 'ID') & (df["translation"] == 'translation')].index
  df = df.drop(rows_to_drop)
  df = df.reset_index(drop=True)

  return df

In [ ]:
df = load_data("validation-00000-of-00001.csv")

In [ ]:
text = df['translation'][0].replace("'", '"')

In [ ]:
df.iloc[5]['translation']

'{\'dyu\': "A y\'a dabla ka yɛlɛ n na.", \'fr\': \'Arrêtez de vous moquer de moi.\'}'

In [ ]:
def process_df(df):
    new_data = {'id':[],'dyu':[],'fr':[]}
    for i in range(len(df)):
        data = df.iloc[i]
        text = data['translation'].replace("'",'"')
        print("text: ",text)
        dic = json.loads(text)
        new_data['id'].append(data['id'])
        new_data['dyu'].append(dic['dyu'])
        new_data['fr'].append(dic['fr'])

    return new_data

In [ ]:
process_df(df)

text:  {"dyu": "I tɔgɔ bi cogodɔ", "fr": "Tu portes un nom de fantaisie."}
text:  {"dyu": "Puɛn saba fɔlɔ.", "fr": "Trois points d’avance."}
text:  {"dyu": "Tile bena", "fr": "Le soleil s’est couché."}
text:  {"dyu": "cogoya kelen", "fr": "Mêmes mouvements."}
text:  {"dyu": "N ma daraka dun ban.", "fr": "Je n’ai pas encore déjeuné."}
text:  {"dyu": "A y"a dabla ka yɛlɛ n na.", "fr": "Arrêtez de vous moquer de moi."}


JSONDecodeError: Expecting ',' delimiter: line 1 column 14 (char 13)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [6]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("uvci/Koumankan_mt_dyu_fr")

In [65]:
CHARS_TO_REMOVE_REGEX = '[!"&\(\),-./:;=?+.\n\[\]]'
SRC_LANG = "dyu"
TRG_LANG = "fr"
PREFIX = "translate Dyula to French:"

In [55]:
import re
def remove_special_characters(text):
    text = re.sub(CHARS_TO_REMOVE_REGEX, " ", text.lower())
    return text.strip()

def clean_text(batch):
    # process source text
    batch["translation"][SRC_LANG] = remove_special_characters(batch["translation"][SRC_LANG])
    # process target text
    batch["translation"][TRG_LANG] = remove_special_characters(batch["translation"][TRG_LANG])
    return batch

ds = ds.map(clean_text)
ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'translation'],
        num_rows: 8065
    })
    validation: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1393
    })
})

In [56]:
# Next Task

#1. load the gpt-2 model and tokenizer, load the base model, not the small model
#2. make sure all the characters does not return <unk> for dyu and french, when tested with the tokenizer, e.g, firstly, get all the characters contain for dyu,
#3. run each character and get the token index, and then decode to see if it successfully decode to the given character, and not <unk>, make sure it happens for all the datasets

In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import pandas as pd

def load_model_and_tokenizer(model_name,tokenizer_name,device_map:str='auto'):

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        )
    model.config.use_cache = False

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,trust_remote_code=True)

    tokenizer.pad_token=tokenizer.eos_token
    tokenizer.padding_side='right'

    return model, tokenizer

#model, tokenizer = load_model_and_tokenizer("openai-community/gpt2", "openai-community/gpt2" )

In [3]:
tokenizer_name = 'openai-community/gpt2'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,trust_remote_code=True)

/home/bello/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [58]:
train_dyu = [data['dyu'] for data in ds['train']['translation']]
test_dyu = [data['dyu'] for data in ds['test']['translation']]
val_dyu = [data['dyu'] for data in ds['validation']['translation']]

train_fr = [data['fr'] for data in ds['train']['translation']]
test_fr = [data['fr'] for data in ds['test']['translation']]
val_fr = [data['fr'] for data in ds['validation']['translation']]

In [62]:
train_df = pd.DataFrame({'dyu':train_dyu,'fr':train_fr})
test_df = pd.DataFrame({'dyu':test_dyu,'fr':test_fr})
val_df = pd.DataFrame({'dyu':val_dyu,'fr':val_fr})

In [64]:
train_df.to_csv('final_train_df.csv',index=False)
test_df.to_csv('final_test_df.csv',index=False)
val_df.to_csv('final_val_df.csv',index=False)

In [65]:
train_df

,dyu,fr
0,A bi ji min na,Il boit de l’eau.
1,A le dalakolontɛ lon bɛ.,Il se plaint toujours.
2,Mun? Fɛn dɔ.,Quoi ? Quelque chose.
3,O bɛ bi bɔra fo Gubeta.,Tous sortent excepté Gubetta.
4,A ale lo bi da bugɔ la!,Ah ! c’est lui… il sonne…
...,...,...
8060,Alu a yi n ka yanmariya kɔnɔ.,Allez… attendez mes ordres.
8061,O fatura aksidan ni na.,Ils ont péri dans l'accident.
8062,An! Ni tericɛ fariman ni.,Ah ! ce brave ami !
8063,Sen bi dougouma,À bas les pattes !


In [66]:
from datasets import Dataset

In [67]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

In [70]:
train_ds.save_to_disk('train_ds')
test_ds.save_to_disk('test_ds')
val_ds.save_to_disk('val_ds')

Saving the dataset (0/1 shards):   0%|          | 0/8065 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1393 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1471 [00:00<?, ? examples/s]

In [71]:
tokenizer

GPT2TokenizerFast(name_or_path='openai-community/gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
text = "<human>: A bi ji min na <assistant>: Il boit de l’eau. <|endoftext|>"

In [86]:
def format(data):
    return {"text":f"<human>: {data['dyu']} <assistant>: {data['fr']}"}

In [92]:
new_train_ds = train_ds.map(lambda x: format(x))
new_test_ds = test_ds.map(lambda x: format(x))
new_val_ds = val_ds.map(lambda x: format(x))

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1393 [00:00<?, ? examples/s]

Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

In [94]:
new_train_ds.save_to_disk('final_train_ds')
new_test_ds.save_to_disk('final_test_ds')
new_val_ds.save_to_disk('final_val_ds')

Saving the dataset (0/1 shards):   0%|          | 0/8065 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1393 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1471 [00:00<?, ? examples/s]

In [77]:
train_dyu[0]

'A bi ji min na'

In [78]:
train_fr[0]

'Il boit de l’eau.'

In [81]:
ds['train'][0]

{'ID': 'ID_18897661270129',
 'translation': {'dyu': 'A bi ji min na', 'fr': 'Il boit de l’eau.'}}

In [27]:
chars = list(set("".join(train_dyu + test_dyu + val_dyu)))

In [30]:
tokenizer.decode(tokenizer.encode('ò'))

'ò'

In [43]:
50256 in tokenizer.encode('<|endoftext|>')

True

In [44]:
unks = []
not_unks = []

for char in chars:
    token = tokenizer.encode(char)
    if 50256 in token:
        unks.append(char)
    else:
        not_unks.append(char)

In [49]:
tokenizer.decode(tokenizer.encode("'"))

"'"

In [133]:
check = [dyu[SRC_LANG] for dyu in ds['train']["translation"]]

In [134]:
tokenizer.decode(tokenizer.encode(check[0]))

'a bi ji min na'

In [135]:
tokenizer.decode(tokenizer.encode_plus(check[0])["input_ids"])

'a bi ji min na'

In [136]:
tokenizer.convert_ids_to_tokens(tokenizer.encode_plus(check[0])["input_ids"])

['a', 'Ġbi', 'Ġj', 'i', 'Ġmin', 'Ġna']

In [137]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<|endoftext|>'}

In [138]:
input = tokenizer(check, max_length=512, padding=True, truncation=True)

In [142]:
token_ids = input['input_ids']
tokens = [tokenizer.convert_ids_to_tokens(token_id) for token_id in token_ids]

unknown_tokens = [token for token in tokens if token == tokenizer.unk_token]
pad_token = [token for token in tokens if token == tokenizer.pad_token]
eos_token = [token for token in tokens if token == tokenizer.eos_token]

print(f"Number of pad tokens: {len(pad_token)}")
print(f"Number of end of sentence tokens: {len(eos_token)}")
print(f"Number of unknown tokens: {len(unknown_tokens)}")

Number of pad tokens: 0
Number of end of sentence tokens: 0
Number of unknown tokens: 0


In [143]:
input.keys()

dict_keys(['input_ids', 'attention_mask'])

In [144]:
inputs = [PREFIX + example[SRC_LANG] for example in ds['train']["translation"]]
targets = [example[TRG_LANG] for example in ds['train']["translation"]]

In [149]:
inputs[0] , targets[0]

('translate Dyula to French:a bi ji min na', 'il boit de l’eau')